In [56]:
import torch
import numpy as np
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torchvision
import torch.nn as nn

In [55]:
def squash(s, dim=-1, epsilon=1e-3):
    norm_s = torch.sqrt(torch.sum(t**2, dim, keepdim=True) + epsilon)
    v = (norm_s / (1 + norm_s)) + (s / norm_s)
    return v

In [123]:
# https://discuss.pytorch.org/t/why-softmax-function-cant-specify-the-dimension-to-operate/2637
def softmax(input, axis=1):
    input_size = input.size()
    
    trans_input = input.transpose(axis, len(input_size)-1)
    trans_size = trans_input.size()

    input_2d = trans_input.contiguous().view(-1, trans_size[-1])
    
    soft_max_2d = nn.functional.softmax(input_2d)
    
    soft_max_nd = soft_max_2d.view(*trans_size)
    return soft_max_nd.transpose(axis, len(input_size)-1)

In [137]:
class CapsuleLayer(nn.Module):
    def __init__(self, in_channels, capsule_dimension, num_capsules, kernel_size=None, routing=False, num_iterations=0, stride=1):
        
        super(CapsuleLayer, self).__init__()
        
        self.in_channels = in_channels
        self.capsule_dimension = capsule_dimension
        self.num_capsules = num_capsules
        self.kernel_size = kernel_size
        self.routing = routing
        self.num_iterations = num_iterations
        self.stride = stride
        
        if not self.routing:
            
            self.conv = nn.Conv2d(self.in_channels, self.capsule_dimension * self.num_capsules,\
                                  self.kernel_size, self.stride)      
            
        
        else:
            
            # todo: to be calculated later
            self.width = 1152
            
            self.weights = nn.Parameter(torch.from_numpy(0.01 * np.random.randn(1, self.width, self.width, self.num_capsules, self.in_channels)))
            
            

        
    def forward(self, x):
        
        if not self.routing:
            
            self.conv_out = self.conv(x)
            
            # todo: to be calculated later
            width = 6 # (W - F + 2 * P) / S + 1
            
            conv2 = self.conv_out.view(-1, width * self.width * self.num_capsules, self.capsule_dimension)
            
            squash_conv = squash(conv2)
            
            return squash_conv
                
                
        else:
            # x -> [batch size , 1152, 8]
             
            batch_size = x.size()[0]
            
            u = x.expand(x.size()[0], x.size()[1], self.num_capsules, x.size()[2], 1)
            
            u_hat = torch.functional.matmul(self.weights, u)
            
            v = None
            
            for r in self.num_iterations:
                
                b = nn.Parameter(torch.zeros(batch_size, x.size()[1], self.num_capsules, 1, 1))
                
                c = softmax(b, axis=2)
                
                s = torch.mm(c, u_hat)
                
                v = squash(s, axis=-2)
                
                b += torch.mm(u_hat, v)
                
            return v
                
            
        

In [138]:
class Net(nn.Module):
    def __init__(self):
        
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 256, 9)
        
        self.primary_capsule = CapsuleLayer(256, 8, 32, kernel_size=9, routing=False, num_iterations=0, stride=2)
        
        self.digital_capsule = CapsuleLayer(8, 16, 10, kernel_size=None, routing=True, num_iterations=10, stride=1)
        
        
        
    def forward(self, x):
        
        x = self.conv1(x)
        
        x = nn.functional.relu(x)
        
        x = self.primary_capsule(x)
        
        x = self.digital_capsule(x)
        
        
        
        
        
        

In [139]:
net = Net()

In [140]:
net

Net (
  (conv1): Conv2d(1, 256, kernel_size=(9, 9), stride=(1, 1))
  (primary_capsule): CapsuleLayer (
    (conv): Conv2d(256, 256, kernel_size=(9, 9), stride=(2, 2))
  )
  (digital_capsule): CapsuleLayer (
  )
)